<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Why" data-toc-modified-id="Why-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Why</a></span></li><li><span><a href="#Doc" data-toc-modified-id="Doc-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Doc</a></span></li><li><span><a href="#Auth" data-toc-modified-id="Auth-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Auth</a></span></li><li><span><a href="#Create" data-toc-modified-id="Create-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create</a></span></li><li><span><a href="#Update" data-toc-modified-id="Update-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Update</a></span></li><li><span><a href="#Read" data-toc-modified-id="Read-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Read</a></span></li><li><span><a href="#Delete" data-toc-modified-id="Delete-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Delete</a></span></li><li><span><a href="#Listen-a-cell" data-toc-modified-id="Listen-a-cell-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Listen a cell</a></span></li><li><span><a href="#Discussion" data-toc-modified-id="Discussion-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Discussion</a></span></li><li><span><a href="#Wrappers" data-toc-modified-id="Wrappers-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Wrappers</a></span></li></ul></div>

# Why

1. 更好的管理 sheets (**label**, eda shared with different team)
2. 每次都要從工作站下載 csv ，再貼上到 good drive，在複製貼上到 sheets 很麻煩

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['SPARK_HOME'] = '/opt/spark/versions/spark-3.1'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/opt/share/secrets/google-application-credential.json"

In [23]:
%matplotlib inline
import pandas as pd
import numpy as np
import json
from os.path import join as PJ
from typing import List
import requests
import io

In [24]:
import pygsheets

In [25]:
pygsheets.__version__

'2.0.5'

# Doc

# Auth

In [26]:
# auth stuff

gc = pygsheets.authorize(service_account_file='/opt/share/secrets/google-application-credential.json')

# gc = pygsheets.authorize(service_account_env_var='GOOGLE_APPLICATION_CREDENTIALS')

In [27]:
print(dir(gc))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'create', 'drive', 'get_range', 'logger', 'oauth', 'open', 'open_all', 'open_as_json', 'open_by_key', 'open_by_url', 'run_batch', 'set_batch_mode', 'sheet', 'spreadsheet_cls', 'spreadsheet_ids', 'spreadsheet_titles', 'teamDriveId']


# Create

Create in drive should be easier

TODO : 

where is the sheets when service account create them?

In [28]:
# It can do that! with some effert.

joe_sh = gc.create(title='Joe_Test')

#  A permission role can only be one of ['organizer', 'owner', 'writer', 'commenter', 'reader'].
# also you can share a domain
joe_sh.share('joetsai@pixnet.tw',role='writer')

print('url of the sheets : ', joe_sh.url)
print('title of the sheets : ', joe_sh.title)

print('class and method : ')

print(joe_sh, dir(joe_sh), sep='\n\n')

# Findings :
# 1. the sheets create by the service account, so you might have no right to check the sheets
# 2. we might wanna add some useful information in __str__ e.g. url, Sheets name



url of the sheets :  https://docs.google.com/spreadsheets/d/1ZTb_Ve2qQGcqifJJ5Omr8Pa8VM3f694NL0nJjwEg9qY
title of the sheets :  Joe_Test
class and method : 
<Spreadsheet 'Joe_Test' Sheets:1>

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_defaultFormat', '_fetch_sheets', '_id', '_jsonsheet', '_named_ranges', '_sheet_list', '_title', 'add_worksheet', 'client', 'create_developer_metadata', 'custom_request', 'default_parse', 'defaultformat', 'del_worksheet', 'delete', 'export', 'fetch_properties', 'find', 'get_developer_metadata', 'id', 'logger', 'named_ranges', 'permissions', 'protected_ranges', 'remove_permission', 'replace', 'share', 'sheet1', 'title', 'to_json', '

# Update

In [29]:
# Open spreadsheet and then worksheet

# return a sheets
# need to setup the crenditial
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=0')

print(type(sh), sh)

# working sheets

wks = sh.sheet1

# Update a cell with value (just to let him know values is updated ;) )
wks.update_value('A1', "Hey yank this numpy array")
my_nparray = np.random.randint(10, size=(3, 4))

# update the sheet with array
wks.update_values('A2', my_nparray.tolist())

# already update the value =)

# https://docs.google.com/spreadsheets/d/1KLzEsuFdx2_9o1Kf7GgR_S-iBQQ20ZDHWpMClKDVtWE/edit#gid=0

<class 'pygsheets.spreadsheet.Spreadsheet'> <Spreadsheet 'test' Sheets:3>


In [30]:
# sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=0')


sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=847289509')

In [31]:
sh.worksheet(property='id', value=847289509).get_as_df()

,note,label,string_id,aonther_id,poi_name,views,click
0,,,1,1,abc,100,3
1,,,2,2,defg,50,1
2,,,3,3,sss,NotaNumber,0


In [32]:
df = (
    pd.DataFrame(
    {
        'note':['','',''],
        'label':['','',''],
        'string_id':['0001','0002','0003'],
        'aonther_id':['1','2','3'],
        'poi_name' : ['abc','defg','sss'],
        'views' : [100, 50, np.nan],
        'click' : [3, 1, 0]
    }
    )
)

try:
    wks = sh.worksheet_by_title('poi_views_v20210804')
except:
    wks = sh.add_worksheet('poi_views_v20210804')
print('Now we have : ',sh.worksheets(), wks, sep='\n\n')

# write dataframe into sheets
wks.set_dataframe(df, start=(0,0), nan='NotaNumber')

print(sh, wks, sh.url, sep='\n\n')
# Findins:

# 1. write dataframe into sheets with ease
# 2. need a work-around about creating sheets
# 3. we might wanna add a sanity check if we wanna wrtie dataframe into a sheet
#    avoiding overwrite the already-labelled data

Now we have : 

[<Worksheet '工作表1' index:0>, <Worksheet 'poi_views_v20210804' index:1>, <Worksheet '工作表2' index:2>]

<Worksheet 'poi_views_v20210804' index:1>
<Spreadsheet 'test' Sheets:3>

<Worksheet 'poi_views_v20210804' index:1>

https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A


In [33]:
wks.__dict__

{'logger': <Logger pygsheets.worksheet (WARNING)>,
 'spreadsheet': <Spreadsheet 'test' Sheets:3>,
 'client': <pygsheets.client.Client at 0x7f449a266e20>,
 '_linked': True,
 'jsonSheet': {'properties': {'sheetId': 847289509,
   'title': 'poi_views_v20210804',
   'index': 1,
   'sheetType': 'GRID',
   'gridProperties': {'rowCount': 100, 'columnCount': 26}}},
 'data_grid': None,
 '_func_calls': [],
 'grid_update_time': None}

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   note        3 non-null      object 
 1   label       3 non-null      object 
 2   string_id   3 non-null      object 
 3   aonther_id  3 non-null      object 
 4   poi_name    3 non-null      object 
 5   views       2 non-null      float64
 6   click       3 non-null      object 
dtypes: float64(1), object(6)
memory usage: 296.0+ bytes


# Read

In [35]:
# you can also do export(csv) and read
# wks.export(xxx.csv), pd_read_csv(xxx.csv)

labelled_df = wks.get_as_df(
    numerize=False # watch out the 0001 will be conver to 1
)

labelled_df.info()
labelled_df

# Findings:
# 1. read sheet into dataframe with ease
# 2. dtype might be different when convert sheets back to dataframe (e.g. 0001 --> 1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   note        3 non-null      object
 1   label       3 non-null      object
 2   string_id   3 non-null      object
 3   aonther_id  3 non-null      object
 4   poi_name    3 non-null      object
 5   views       3 non-null      object
 6   click       3 non-null      object
dtypes: object(7)
memory usage: 296.0+ bytes


,note,label,string_id,aonther_id,poi_name,views,click
0,,,1,1,abc,100,3
1,,,2,2,defg,50,1
2,,,3,3,sss,NotaNumber,0


# Delete

In [36]:
sh.del_worksheet(wks)
# Findings : 
# 1. be careful, it will delete the sheets directly, not a safe-delete(e.g. move into the trash-can)
# 2. we might wanna write a wrapper about safe-delete

# Listen a cell 

In [ ]:
pass

# Discussion


1. `pygsheets` could be a prototype to helps us manage the sheets


2. Use case : 

    - [x] 經常使用 sheets 跨團隊協作，例如共同資料探索，收取 Domain Expert 意見、收取 Label 等
    - [ ] 監聽某個 cell - 作為 trigger
    - [ ] 

3. Meta data(sheets url as data source)

    - 同個 projects 每一次的 label data, schema 可能不太一樣 --> poi_article_mapping 每一次做資料驗證時，關注的重點可能皆不同(poi_name, article_title, or poi_hub, article_for_dummies) --> schema free?

# Wrappers

In [37]:
def read_spreadsheet_to_pandas_v0(sheet_id : str,
                                  gid : int = 0,
                                  sheet_view : bool = False,
                                  fillna=None,
                                  **kwargs):
    """Download google spreadsheet as pandas.DataFrame
    Args:
        sheet_id: The hash id in a google spreadsheet's url. (e.g. 1evuGqCk3B7VWez1JAw249YjIG3rKznDFHz8I_0A64KY)
        fillna: The value that will replace NaN values in the spreadsheet, default is None.
        **kwargs: Users can add settings for pandas to read csv into dataframe. (e.g. pandas's schema dtype=dtype)"""

    download_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
    
    if sheet_view:
        print(f"https://docs.google.com/spreadsheets/d/{sheet_id}/edit#gid={gid}")
    
    s = requests.get(download_url).content
    df = pd.read_csv(io.StringIO(s.decode("utf-8")), **kwargs)

    if fillna is not None:
        df = df.fillna(fillna)

    return df


In [38]:
# Read sheets as dataframe
# Input sheets_id, gid
# Remember to do auth stuff

# gc = pygsheets.authorize(service_account_file='/opt/share/secrets/google-application-credential.json')


# def read_spreadsheet_to_pandas(sheet_id : str, gid : int = 0, fillna=None,
#                                key : str = '/opt/share/secrets/google-application-credential.json'):
#     """Download google spreadsheet as pandas.DataFrame
#     Args:
#         sheet_id: The hash id in a google spreadsheet's url. (e.g. 1evuGqCk3B7VWez1JAw249YjIG3rKznDFHz8I_0A64KY)
        
#         fillna: The value that will replace NaN values in the spreadsheet, default is None.
#         **kwargs: Users can add settings for pandas to read csv into dataframe. (e.g. pandas's schema dtype=dtype)"""

# #     url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
#     url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/edit#gid={gid}"
#     print(url)
    
#     gc = pygsheets.authorize(service_account_file=key)
    
#     df = (
#         gc.open_by_url(url)
#         .worksheet(property='id',value=gid)
#         .get_as_df(numerize=False) # make sure "0001" will not converted to 1
#          )

#     if fillna is not None:
#         df = df.fillna(fillna)

#     return df


In [39]:
# https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438
    
res = read_spreadsheet_to_pandas(sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',gid=1362269438)

NameError: name 'read_spreadsheet_to_pandas' is not defined

In [ ]:
res = read_spreadsheet_to_pandas_v0(sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
                                    gid=1362269438,
                                    sheet_view=True
                                   )

In [ ]:
display(res)
type(res)

In [ ]:
# pip install pygsheets==2.0.5
def write_pandas_to_spreadsheet(pdf : pd.DataFrame,
                                sheet_id : str,
                                gid: int = 0,
                                mode : str = 'append',
                                key : str = '/opt/share/secrets/google-application-credential.json',
                                **kwargs) -> dict:
    """
    return status and error message
    mode : append or overwrite
    """
    assert mode in ['append','overwrite'], f"write mode should in ['append','overwrite'], you mode {mode}"
    share_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/edit#gid={gid}"
    
    gc = pygsheets.authorize(service_account_file=key)
    wks = gc.open_by_url(share_url).worksheet(property='id',value=gid)
    res = {}
    if mode == 'overwrite':
        try:
            wks.clear()
            wks.set_dataframe(pdf, start=(0,0), **kwargs)
            res['error'] = False
            res['message'] = f'{mode} dataframe to {share_url} successfully'
        except Exception as e:
            res['error'] = True
            res['message'] = f'ERROR : {e} when writing {share_url}'
    elif mode =='append':
        raise NotImplementedError
            
    return res

In [ ]:
df = (
    pd.DataFrame(
    {
        'note':['','',''],
        'label':['','',''],
        'string_id':['0001','0002','0003'],
        'aonther_id':['1','2','3'],
        'poi_name' : ['abc','defg','sss'],
        'views' : [100, 50, np.nan],
        'click' : [3, 1, 0]
    }
    )
)

In [ ]:
write_pandas_to_spreadsheet(
    df,
    sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
    gid=1362269438,
    mode='overwrite')

In [ ]:
(
    gc.open_by_url('https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438')
    .worksheet(property='id',value=1362269438)
    .get_as_df()
)

In [40]:
from pixlake.catadog.models.spreadsheet import Spreadsheet
from pixlake.catadog.models.spark import DatePartitionDir

In [41]:
s1=Spreadsheet(
    description='abc',
    name = 'sheet',
    sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A',
    gid = 1362269438
#     drescription: str,
)

In [42]:
print(dir(s1))

['__annotations__', '__attrs_attrs__', '__attrs_post_init__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', 'description', 'download_url', 'dump', 'evolve', 'format', 'from_dict', 'from_json', 'gid', 'identifier', 'load', 'mutate_from', 'name', 'schema', 'share_url', 'sheet_id', 'to_dict', 'to_json']


In [44]:
pdf = s1.load()

pdf

,note,label,string_id,aonther_id,poi_name,views,click
0,NaN,NaN,1,1,abc,100.0,3
1,NaN,NaN,2,2,defg,50.0,1
2,NaN,NaN,3,3,sss,NaN,0


In [45]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   note        0 non-null      float64
 1   label       0 non-null      float64
 2   string_id   3 non-null      int64  
 3   aonther_id  3 non-null      int64  
 4   poi_name    3 non-null      object 
 5   views       2 non-null      float64
 6   click       3 non-null      int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 296.0+ bytes


In [46]:
s1.dump(pdf.fillna(''), mode='overwrite')

Already export dataframe to :  https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438


Spreadsheet(sheet_id='1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A', gid=1362269438, name='sheet', description='abc', schema={}, identifier='', share_url='https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/edit#gid=1362269438', download_url='https://docs.google.com/spreadsheets/d/1Hmm_ssu4NvzF_qH03pe288KMVPJoN4jYlEhlIiKFF5A/export?format=csv&gid=1362269438')

In [49]:
DatePartitionDir(identifier='',prefix='bv',basename='a').basename

'a'